### CMSC 673 Intro to NLP Final Project 
#### Patty Delafuente and Neel Patel
Work split:
Patty - approach and model training, proposal and document editing, inferencing and next step  
Neel - First paper draft, approach review and model evaluation, review  
Date: Dec 20, 2023

In [1]:
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md
import torch
MAX_LEN = 512 
roberta_checkpoint = "roberta-large"


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#dataprep
import pandas as pd
import os
#use 

#train_df=pd.read_csv('df_train.csv')
train_df = pd.read_csv('training_data.csv')
#val_df=pd.read_csv('df_val.csv')
test_df=pd.read_csv('df_test.csv')
# dummy target column for merge test and train into one huggingface data
#test
test_df['target'] = 0 
train_df.info()
#val_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6742 entries, 0 to 6741
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  6742 non-null   object
 1   target    6742 non-null   int64 
 2   idx       6742 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 158.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1062 non-null   object
 1   target    1062 non-null   int64 
 2   idx       1062 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 25.0+ KB


In [4]:
train_df.head()
#target is the trp label, I just labeled everything with a question mark as 1


,sentence,target,idx
0,One more pseudo generalization and I'm giving up.,0,1
1,One more pseudo generalization or I'm giving up.,0,2
2,"The more we study verbs, the crazier they get.",0,3
3,Day by day the facts are getting murkier.,0,4
4,I'll fix you a drink.,0,5


In [5]:
#Patty
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md

#unable to use validation dataset I created - issue with huggingface (or my brain) format. Instead, the training dataset will be split
from datasets import Dataset
#Convert the training dataframe to HuggingFace dataset
dataset = Dataset.from_pandas(train_df)
#Split the dataset into training and validation datasets
data = dataset.train_test_split(train_size=0.8, seed=42)
#Rename the default "test" split to "validation"
data['val'] = data.pop("test")
#Convert the test dataframe to HuggingFace dataset and add it into the first dataset
data['test'] = Dataset.from_pandas(test_df)
#Save
data.save_to_disk("processed_hf")

Saving the dataset (0/1 shards):   0%|          | 0/5393 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1349 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1062 [00:00<?, ? examples/s]

In [6]:
#load roberta tokenizer
from transformers import AutoTokenizer
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_checkpoint, add_prefix_space=True)

In [7]:
def roberta_preprocessing_function(examples):
    return roberta_tokenizer(examples['sentence'], truncation=True, max_length=MAX_LEN)

In [8]:
roberta_preprocessing_function(data['train'][0])

{'input_ids': [0, 20, 2335, 439, 5373, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [9]:
#Patty 
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md

#apply to entire dataset
col_to_delete = ['idx', 'sentence']
#Apply the preprocessing function
roberta_tokenized_datasets = data.map(roberta_preprocessing_function, batched=False)
#Remove the undesired columns
roberta_tokenized_datasets = roberta_tokenized_datasets.remove_columns(col_to_delete)
#Rename the target to label as for HugginFace standards
roberta_tokenized_datasets = roberta_tokenized_datasets.rename_column("target", "label")
#Set to torch format
roberta_tokenized_datasets.set_format("torch")

Map:   0%|          | 0/5393 [00:00<?, ? examples/s]

Map:   0%|          | 0/1349 [00:00<?, ? examples/s]

Map:   0%|          | 0/1062 [00:00<?, ? examples/s]

In [10]:
print(roberta_tokenized_datasets['train'][1])

{'label': tensor(1), 'input_ids': tensor([    0,  3394,   222,    47,   972,    70,    77,    47,    58,    11,
          211, 11228,   116,     2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


In [11]:
# Data collator for padding a batch of examples to the maximum length seen in the batch
from transformers import DataCollatorWithPadding
roberta_data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)

In [12]:
from transformers import AutoModelForSequenceClassification 
# Load a pre-trained model with a sequence classification header 
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_checkpoint, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
print(roberta_tokenized_datasets['train'][1])
print(roberta_tokenized_datasets['val'][1])
print(roberta_tokenized_datasets['test'][1])

{'label': tensor(1), 'input_ids': tensor([    0,  3394,   222,    47,   972,    70,    77,    47,    58,    11,
          211, 11228,   116,     2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'label': tensor(0), 'input_ids': tensor([    0,    38,    33,    10,  2157,    14, 10832,    40,   311,    62,
            4,     2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}
{'label': tensor(0), 'input_ids': tensor([    0,  1585,  3148,  3268,   160,     5, 26711,     4,     2]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1])}


In [14]:
#Patty
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md

#For generating the training batches, we also need to pad the rows of a given batch to the same maximum length, the maximum length found in this batch.
#For that, we will use the DataCollatorWithPadding class:
# Data collator for padding a batch of examples to the maximum length seen in the batch
from transformers import DataCollatorWithPadding
# roberta_data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)
roberta_data_collator = DataCollatorWithPadding(
    tokenizer=roberta_tokenizer,
    padding="max_length",  # or any other appropriate padding strategy
    max_length=MAX_LEN,
    return_tensors="pt",  # Ensure the output is PyTorch tensors
)

In [15]:
from transformers import AutoModelForSequenceClassification
# Load a pre-trained model with a sequence classification header
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_checkpoint, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
#Patty
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md

#lora setup for Roberta classifier
from peft import get_peft_model, LoraConfig, TaskType
roberta_peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=2, lora_alpha=16, lora_dropout=0.1, bias="none",
)
roberta_model = get_peft_model(roberta_model, roberta_peft_config)
roberta_model.print_trainable_parameters()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


trainable params: 1,248,258 || all params: 356,610,052 || trainable%: 0.35003444042009224


In [17]:
#Neel
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md

#evaluation metrics
#Define evaluation metrics
import evaluate
import numpy as np
def compute_metrics(eval_pred):
    # All metrics are already predefined in the HF `evaluate` package
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    f1_metric= evaluate.load("f1")
    accuracy_metric = evaluate.load("accuracy")

    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    # print(logits)
    # print(labels)
    # print(logits, len(logits))
    # print(labels, len(labels))
    # predictions = np.argmax(logits, axis=-1)
    # predictions = [predictions]
    # labels = [labels]
    predictions = logits
    print('predictions:', predictions)
    print('labels:', labels)
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    # The trainer is expecting a dictionary where the keys are the metrics names and the values are the scores.
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [18]:
#Patty
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md

#trainer setup Roberta

roberta_model= torch.nn.DataParallel(roberta_model)
roberta_model = roberta_model.module  # Access the underlying model from DataParallel
roberta_model.to(device)


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(
                    in_features=1024, out_features=1024, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                

In [19]:
#Patty
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md

#set training arguments
from transformers import TrainingArguments, Trainer

lr = 1e-4
batch_size = 8
num_epochs = 5

training_args = TrainingArguments(
    output_dir="roberta-large-lora-token-classification",
    learning_rate=lr,
    lr_scheduler_type= "constant",
    warmup_ratio= 0.1,
    max_grad_norm= 0.3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.001,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #report_to="wandb",
    fp16=False,
    gradient_checkpointing=True,
)

In [20]:
#Patty
# adapted work from https://github.com/mehdiir/Roberta-Llama-Mistral/blob/main/Lora-for-sequence-classification-with-Roberta-Llama-Mistral.md

from torch.utils.data import DataLoader

roberta_trainer = Trainer(
    model=roberta_model,
    args=training_args,
    train_dataset=roberta_tokenized_datasets['train'],
    eval_dataset=roberta_tokenized_datasets['val'],
    data_collator=roberta_data_collator,
    )

data_loader = DataLoader(roberta_tokenized_datasets['train'], batch_size=1, shuffle=True)
print(data_loader)
print(roberta_tokenized_datasets['train'][0].keys())
print(roberta_data_collator)

for batch in data_loader:
  print(batch)
  print(batch.keys())
  break

    # Start training
roberta_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


dict_keys(['label', 'input_ids', 'attention_mask'])
DataCollatorWithPadding(tokenizer=RobertaTokenizerFast(name_or_path='roberta-large', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}, padding='max_le

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors we

Epoch,Training Loss,Validation Loss
1,No log,0.053212
2,0.080900,0.043433
3,0.034100,0.029606
4,0.034100,0.027315
5,0.026100,0.026402


/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: 

TrainOutput(global_step=1690, training_loss=0.04441779943612906, metrics={'train_runtime': 667.7067, 'train_samples_per_second': 40.384, 'train_steps_per_second': 2.531, 'total_flos': 2.523293051099136e+16, 'train_loss': 0.04441779943612906, 'epoch': 5.0})

In [22]:
##Neel
from tqdm.auto import tqdm
from transformers import RobertaTokenizer
from torch.utils.data import DataLoader

checkpoint_dir = 'roberta-large-lora-token-classification/checkpoint-1690/'
loaded_model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir)
# loaded_tokenizer = AutoTokenizer.from_pretrained(model_dir)
# loaded_tokenizer = RobertaTokenizer.from_pretrained(model_dir)

# Data collator for padding a batch of examples to the maximum length
# test_data_collator = DataCollatorWithPadding(
#     tokenizer=loaded_tokenizer,
#     padding="max_length",
#     max_length=MAX_LEN,
#     return_tensors="pt",
# )

loaded_model = loaded_model.to(device)
test_dataset = roberta_tokenized_datasets['val']
test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

all_labels = []
all_predictions = []

for batch in tqdm(test_data_loader, desc="Evaluating"):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["label"].to(device)

    with torch.no_grad():
        outputs = loaded_model(input_ids, attention_mask=attention_mask, labels=labels)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    all_labels.extend(labels.cpu().numpy())
    all_predictions.extend(predictions.cpu().numpy())

# Convert to numpy arrays
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Compute evaluation metrics using the provided compute_metrics function
evaluation_metrics = compute_metrics((all_predictions, all_labels))

# Display evaluation metrics
print("Evaluation Metrics:")
for metric_name, metric_value in evaluation_metrics.items():
    print(f"{metric_name}: {metric_value}")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluating:   0%|          | 0/1349 [00:00<?, ?it/s]

predictions: [0 0 0 ... 0 0 0]
labels: [0 0 0 ... 0 0 0]
Evaluation Metrics:
precision: 0.9583333333333334
recall: 0.989247311827957
f1-score: 0.9735449735449735
accuracy: 0.9962935507783544


In [27]:
##Patty
from transformers import pipeline
#from langchain.llms import HuggingFacePipeline
trp_pipe = pipeline("text-classification", model=checkpoint_dir)
text = "What are you doing for the holidays?"
trp_score = trp_pipe(text)[0]['score']
print("The chance we should switch speakers after this sentence is ",trp_score)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The chance we should switch speakers after this sentence is  0.6000781059265137


In [31]:
text = "Do you want to watch a movie?"
trp_score = trp_pipe(text)[0]['score']
print("The chance we should switch speakers after this sentence is ",trp_score)

The chance we should switch speakers after this sentence is  0.5964725017547607
